# DA3-A3 
## Parker & Co. -1
Peter Szilvasi,
Máté Schieszler 

In [30]:
import pandas as pd
import datetime
import numpy as np

In [31]:
original_data = pd.read_csv('https://raw.githubusercontent.com/mateschieszler/DA3-A3/main/cs_bisnode_panel.csv')

# Construction of the holdout set

We are only interested in predicting default for ‘ind2 == 26’, which is the selected industry, and the
firm is a small or medium enterprise (SME). Thus, yearly sales in 2014 was between 1000 EUR and 10
million EUR.

Existed in 2014 (sales > 0), but did not exists in 2015 (sales is 0 or missing)

• If you do the sample design properly, you have an overall of 1037 firms. 56 firms defaulted, and 981
stayed alive. The average sales of the firms is 0.4902 million EUR, with the minimum of 0.00107 million
EUR and 9.57648 million EUR.

In [32]:
# copy original so we don't have to reload it every time (it takes more than 10s)
data = original_data.copy()

In [33]:
# filter out ind2 != 26
data = data.loc[data['ind2'] == 26]
data['sales'].fillna(0, inplace=True)
data.shape

(14877, 48)

In [34]:
# add all missing year and comp_id combinations -
# originally missing combinations will have NAs in all other columns
data = (
    data.set_index(["year", "comp_id"])
    .unstack(fill_value="toReplace")
    .stack()
    .reset_index()
)
data = data.replace("toReplace", np.nan)

In [35]:
# generate status_alive; if sales larger than zero and not-NA, then firm is alive
data["status_alive"] = (data["sales"] > 0 & (False == data["sales"].isna())).astype(int)

# defaults in one year if there are sales in this year but no sales the next year
# Status_in_a_year: data.groupby('comp_id')['status_alive'].shift(-1)
data["default"] = (
    (data["status_alive"] == 1)
    & (data.groupby("comp_id")["status_alive"].shift(-1) == 0)
).astype(int)

In [36]:
holdout_set = data.loc[(data['year'] == 2014) & (data['sales'] >= 1000) & (data['sales'] <= 10000000)]

# Display information about the resulting holdout set
print("Total firms in holdout set:", len(holdout_set))
print("Number of defaulted firms:", len(holdout_set[holdout_set['default'] == 1]))
print("Number of firms that stayed alive:", len(holdout_set[holdout_set['default'] == 0]))  # Since all firms meet the condition
print("Average sales of the firms:", holdout_set['sales'].mean())
print("Minimum sales:", holdout_set['sales'].min())
print("Maximum sales:", holdout_set['sales'].max())

Total firms in holdout set: 1037
Number of defaulted firms: 56
Number of firms that stayed alive: 981
Average sales of the firms: 490202.21792682
Minimum sales: 1070.370361328125
Maximum sales: 9576485.0


In [37]:
data.shape

(21948, 50)

In [38]:
# with this -> data = working set
data = data.drop(holdout_set.index)

# Data Exploration

In [39]:
original_data.shape

(287829, 48)

In [40]:
data.shape

(20911, 50)

In [41]:
data["default"].describe()

count    20911.000000
mean         0.062790
std          0.242591
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: default, dtype: float64

In [42]:
data["sales"].describe()

count    1.384000e+04
mean     1.481976e+06
std      7.016217e+06
min     -8.149259e+04
25%      9.292592e+03
50%      5.551111e+04
75%      2.753981e+05
max      1.107458e+08
Name: sales, dtype: float64

In [50]:
data["year"].value_counts()

2005    1829
2006    1829
2007    1829
2008    1829
2009    1829
2010    1829
2011    1829
2012    1829
2013    1829
2015    1829
2016    1829
2014     792
Name: year, dtype: int64

In [51]:
original_data["year"].value_counts()

2013    28474
2012    28469
2011    28222
2014    28151
2015    27506
2010    26784
2009    25596
2008    23733
2007    21804
2006    20200
2005    19147
2016     9743
Name: year, dtype: int64

In [43]:
# look at missing values
data.isna().sum()

year                        0
comp_id                     0
begin                    7071
end                      7071
COGS                    19145
amort                    7357
curr_assets              7076
curr_liab                7076
extra_exp                7941
extra_inc                7941
extra_profit_loss        7892
finished_prod           19175
fixed_assets             7076
inc_bef_tax              7304
intang_assets            7076
inventories              7076
liq_assets               7076
material_exp             7357
net_dom_sales           19145
net_exp_sales           19145
personnel_exp            7357
profit_loss_year         7664
sales                    7071
share_eq                 7076
subscribed_cap           7076
tang_assets              7125
wages                   19182
D                       20911
balsheet_flag            7071
balsheet_length          7071
balsheet_notfullyear     7071
founded_year             9684
exit_year               19532
ceo_count 

In [44]:
# Calculate percentage of missing values in each column of the original dataset
original_missing_percentage = (original_data.isnull().sum() / len(original_data)) * 100

# Calculate percentage of missing values in each column of the filtered dataset
filtered_missing_percentage = (data.isnull().sum() / len(data)) * 100

# Calculate percentage of missing values in each column of the filtered dataset
filtered_missing_percentage = (data.isnull().sum() / len(data)) * 100

# Combine the missing percentage information into a DataFrame for easier comparison
missing_info = pd.DataFrame({
    'Original Data Missing %': original_missing_percentage,
    'Filtered Data Missing %': filtered_missing_percentage
})

print(missing_info)

                      Original Data Missing %  Filtered Data Missing %
COGS                                93.656998                91.554684
D                                  100.000000               100.000000
amort                                2.793325                35.182440
balsheet_flag                        0.000000                33.814739
balsheet_length                      0.000000                33.814739
balsheet_notfullyear                 0.000000                33.814739
begin                                0.000000                33.814739
birth_year                          38.848761                59.992349
ceo_count                           19.604348                46.310554
comp_id                              0.000000                 0.000000
curr_assets                          0.045513                33.838650
curr_liab                            0.045513                33.838650
default                                   NaN                 0.000000
end   

Would it makes sense to examine all original data as it might have some additinal info which might uncover relations that are not present in our data?

In [45]:
# drop columns with over 12000 missing values
to_drop = ["COGS","finished_prod","net_dom_sales","net_exp_sales","wages","D","exit_year","birth_year","exit_date","labor_avg"]
data = data.drop(to_drop, axis=1)

In [46]:
data

,year,comp_id,begin,end,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,status_alive,default
0,2005,6.538183e+06,2005-01-01,2005-12-31,792.592590,6237.037109,348.148163,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2.0,East,1992-08-25,1,0
1,2005,6.934257e+06,2005-05-13,2005-12-31,803.703674,4648.147949,9311.111328,0.000000,0.0,0.000000,...,NaN,NaN,2660.0,26.0,2.0,3.0,East,2005-04-26,1,1
2,2005,8.416055e+06,2005-01-01,2005-12-31,3155.555664,71070.367188,25514.814453,74.074074,0.0,-74.074074,...,female,Domestic,2651.0,26.0,2.0,1.0,Central,1995-08-28,1,0
3,2005,9.283375e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,2005,1.089394e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,2016,4.604901e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
21944,2016,4.610112e+11,2016-01-01,2016-12-31,148.148148,67448.148438,5059.259277,NaN,NaN,NaN,...,male,Domestic,2620.0,26.0,2.0,3.0,Central,1997-12-10,1,0
21945,2016,4.620594e+11,2016-01-01,2016-12-31,559.259277,64011.109375,8662.962891,NaN,NaN,NaN,...,male,Domestic,2630.0,26.0,2.0,1.0,Central,1996-12-30,1,0
21946,2016,4.623300e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [47]:
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [48]:
(ggplot(data, aes(x='COGS'))
+ geom_bar()
)


PlotnineError: "Could not evaluate the 'x' mapping: 'COGS' (original error: name 'COGS' is not defined)"